In [9]:
import pandas as pd
import os
import re
import string
import nltk
from nltk.corpus import stopwords

# Türkçe stopwords (gereksiz kelimeler) listesini indir
nltk.download('stopwords')
stop_words = set(stopwords.words('turkish'))

# Ödev analizini bozabilecek ek kelimeleri listeye ekleyelim
ekstra_kelimeler = {'bir', 've', 'ile', 'da', 'de', 'için', 'bu', 'çok', 'mı', 'mi', 'aldım', 'gelen'}
stop_words.update(ekstra_kelimeler)

print("✅ NLP araçları ve Türkçe stopwords listesi hazır.")

✅ NLP araçları ve Türkçe stopwords listesi hazır.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# Klasördeki tüm excel dosyalarını listele
excel_dosyalari = [f for f in os.listdir('.') if f.endswith('.xlsx')]
ham_veriler = []

for dosya in excel_dosyalari:
    try:
        df_temp = pd.read_excel(dosya)
        
        # Kategori ismini dosya adından çek (arcelik_klima -> Klima)
        kategori = dosya.replace('arcelik_', '').split('_')[0].capitalize()
        if 'twitter' in dosya.lower() or '_x_' in dosya.lower(): kategori = 'X-Twitter'

        # Metin sütununu tespit et (aciklama veya Şikayet Mesajı)
        sutun = next((c for c in ['aciklama', 'Şikayet Mesajı', 'sikayet_metni'] if c in df_temp.columns), None)
        
        if sutun:
            for metin in df_temp[sutun]:
                ham_veriler.append({'Kategori': kategori, 'Ham_Metin': str(metin)})
            print(f"✔️ {dosya} sisteme yüklendi.")
    except Exception as e:
        print(f"❌ {dosya} okunurken hata oluştu: {e}")

master_df = pd.DataFrame(ham_veriler)
print(f"\n📊 Toplam ham satır sayısı: {len(master_df)}")

✔️ arcelik_bulasik_makinesi_final.xlsx sisteme yüklendi.
✔️ arcelik_buzdolabi_final.xlsx sisteme yüklendi.
✔️ arcelik_camasir_makinesi_YENI.xlsx sisteme yüklendi.
✔️ arcelik_derin_dondurucu_final.xlsx sisteme yüklendi.
✔️ arcelik_dikey_supurge_final.xlsx sisteme yüklendi.
✔️ arcelik_klima_final.xlsx sisteme yüklendi.
✔️ arcelik_kurutma_makinesi_YENI_FINAL.xlsx sisteme yüklendi.
✔️ arcelik_X_1200_Sikayet_Analiz_Hazir.xlsx sisteme yüklendi.

📊 Toplam ham satır sayısı: 13533


In [11]:
print("🧹 Temizlik operasyonu başlıyor...")

# 1. "HATA: Alınamadı." içeren satırları sil
master_df = master_df[~master_df['Ham_Metin'].str.contains("HATA: Alınamadı.", na=False)]

# 2. Tamamen boş olan satırları sil
master_df = master_df.dropna(subset=['Ham_Metin'])
master_df = master_df[master_df['Ham_Metin'].str.strip() != ""]

# 3. Mükerrer (aynı) şikayetleri sil (X ve Şikayetvar'da çakışanlar için)
master_df = master_df.drop_duplicates(subset=['Ham_Metin'])

print(f"✨ Hatalar ve boşlar temizlendi. Kalan temiz veri: {len(master_df)}")

🧹 Temizlik operasyonu başlıyor...
✨ Hatalar ve boşlar temizlendi. Kalan temiz veri: 12410


In [12]:
def nlp_onislem(metin):
    # Küçük harfe çevir
    metin = metin.lower()
    # URL ve Twitter kullanıcı adlarını sil
    metin = re.sub(r'http\S+|www\S+|@\w+', '', metin)
    # Sayıları ve noktalama işaretlerini sil
    metin = re.sub(r'\d+', '', metin)
    metin = metin.translate(str.maketrans('', '', string.punctuation))
    
    # Kelimelere ayır ve Stopwords/Kısa kelimeleri temizle
    kelimeler = metin.split()
    temiz_kelimeler = [k for k in kelimeler if k not in stop_words and len(k) > 2]
    
    return " ".join(temiz_kelimeler)

print("📝 Metin madenciliği önişlemesi yapılıyor...")
master_df['Temiz_Metin'] = master_df['Ham_Metin'].apply(nlp_onislem)

# Çok kısa (4 kelimeden az) kalan anlamsız metinleri son kez filtrele
master_df = master_df[master_df['Temiz_Metin'].str.split().str.len() > 3]

# Analize hazır veriyi Excel formatında kaydet
# Not: openpyxl kütüphanesi yüklü olmalıdır (pip install openpyxl)
master_df.to_excel("arcelik_master_analiz_hazir.xlsx", index=False)

print("💾 Dosya Excel olarak kaydedildi: arcelik_master_analiz_hazir.xlsx")
print("\n--- ÖDEV İÇİN VERİ ÖZETİ ---")
print(f"Toplam Benzersiz Şikayet: {len(master_df)}")
print("\nKategori Bazlı Dağılım:")
print(master_df['Kategori'].value_counts())

# Excel dosyasının ilk 5 satırını ekranda göster
master_df.head()

📝 Metin madenciliği önişlemesi yapılıyor...
💾 Dosya Excel olarak kaydedildi: arcelik_master_analiz_hazir.xlsx

--- ÖDEV İÇİN VERİ ÖZETİ ---
Toplam Benzersiz Şikayet: 12314

Kategori Bazlı Dağılım:
Kategori
Camasir      2744
Buzdolabi    2524
Klima        2039
Bulasik      1971
Kurutma      1135
X-Twitter    1110
Dikey         426
Derin         365
Name: count, dtype: int64


,Kategori,Ham_Metin,Temiz_Metin
0,Bulasik,"30.12.2025 tarihinde İzmir İstinyePark Arçelik bayisinden 6444 o ki model bulaşık makinesi satın aldık.Kurulumu Karabağlar servisi yaptı. Kurulumdan yaklaşık iki gün sonra eşim bulaşık yıkamak için makinenin kapağını açtığında, kapağın iç kısmının altında vuruk olduğunu fark etti. Hemen servisi aradık, gelip incelediklerinde sadece bir değil, iki adet vuruk olduğunu tespit ettiler ve ürünün fotoğraflarını çekip Arçelik merkeze gönderdiler.Biz de çağrı merkezini arayıp durumu detaylı şekilde bildirerek ayıplı ürünün değiştirilmesini talep ettik. Yaklaşık iki gün sonra servis bizi arayıp, kapağı değiştirebileceklerini ve garanti süresini 3 yıldan 5 yıla çıkarabileceklerini söylediler. Ben bu teklifi kabul etmeyince Tüketici Hakem Heyeti'ne başvurabileceğimizi ifade ettiler. Buna rağmen talebimizin ürün değişimi olduğunu açıkça belirtmemize karşın, 4–5 günlük bir beklemenin ardından ayıplı ürünü değiştirmeyi kabul etmedikleri tarafıma bildirildi.Kapağın iç kısmındaki bu vurukların, bizim kullanımımızla oluşmasının mümkün olmadığını, ancak dışarıdan sert bir darbeyle, adeta çekiçle vurulmuş gibi bir hasar olduğunu düşünüyorum. Bu nedenle ürünü baştan itibaren ayıplı olarak değerlendiriyorum ve yeni, hatasız bir bulaşık makinesi ile değişimini talep ediyorum. 507193906 numaralı servis kaydımın incelenerek yazılı ve kalıcı bir çözüm sunulmasını, ayıplı ürünün yenisiyle değiştirilmesini istiyorum.",tarihinde i̇zmir i̇stinyepark arçelik bayisinden model bulaşık makinesi satın aldıkkurulumu karabağlar servisi yaptı kurulumdan yaklaşık iki gün sonra eşim bulaşık yıkamak makinenin kapağını açtığında kapağın kısmının altında vuruk olduğunu fark etti hemen servisi aradık gelip incelediklerinde sadece değil iki adet vuruk olduğunu tespit ettiler ürünün fotoğraflarını çekip arçelik merkeze gönderdilerbiz çağrı merkezini arayıp durumu detaylı şekilde bildirerek ayıplı ürünün değiştirilmesini talep ettik yaklaşık iki gün sonra servis bizi arayıp kapağı değiştirebileceklerini garanti süresini yıldan yıla çıkarabileceklerini söylediler ben teklifi kabul etmeyince tüketici hakem heyetine başvurabileceğimizi ifade ettiler buna rağmen talebimizin ürün değişimi olduğunu açıkça belirtmemize karşın günlük beklemenin ardından ayıplı ürünü değiştirmeyi kabul etmedikleri tarafıma bildirildikapağın kısmındaki vurukların bizim kullanımımızla oluşmasının mümkün olmadığını ancak dışarıdan sert darbeyle adeta çekiçle vurulmuş hasar olduğunu düşünüyorum nedenle ürünü baştan itibaren ayıplı olarak değerlendiriyorum yeni hatasız bulaşık makinesi değişimini talep ediyorum numaralı servis kaydımın incelenerek yazılı kalıcı çözüm sunulmasını ayıplı ürünün yenisiyle değiştirilmesini istiyorum
1,Bulasik,"Arçelik 6055 uhı model bulaşık makinesini yaklaşık 2 yıldır kullanıyoruz ve ürünümüz hâlâgarantikapsamında. 17.01.2026 tarihinde makine aniden durdu ve şu an hiç çalışmıyor, fişi çekilmiş şekilde bekliyor. Bunun üzerine Arçelik servisini aradığımızda bize ancak 19.01.2026 tarihi saat 15.00’e randevu verilebileceği söylendi.İki gün sonrasına randevu verilmesini, Arçelik gibi bir firmaya yakıştıramadım. Bulaşık makinesinin hayatımızdaki önemini ve şu an içinde bulunduğumuz mağduriyeti dikkate aldığımızda, verilen hizmeti yeterli bulmuyoruz. Bu süreçte gerçekten çok zor durumda kaldık ve bu yaklaşım nedeniyle bir daha Arçelik markasından alışveriş yapmayı düşünmüyoruz.Yaşadığımız mağduriyetin dikkate alınmasını, servis ve randevu süreçlerinizin gözden geçirilmesini ve benzer durumlarda müşterilerin bu şekilde bekletilmemesini talep ediyorum.",arçelik uhı model bulaşık makinesini yaklaşık yıldır kullanıyoruz ürünümüz hâlâgarantikapsamında tarihinde makine aniden durdu çalışmıyor fişi çekilmiş şekilde bekliyor bunun üzerine arçelik servisini aradığımızda bize ancak tarihi saat randevu verilebileceği söylendii̇ki gün sonrasına randevu verilmesini arçelik firmaya yakıştıramadım bulaşık makinesinin hayatımızdaki önemini içinde bulun

In [13]:
# Sadece Kategori ve Temiz_Metin sütunlarını alarak Excel'e kaydet
output_df = master_df[['Kategori', 'Temiz_Metin']]

output_df.to_excel("arcelik_master_temiz_analiz.xlsx", index=False)

print("💾 İşlem Başarılı! 'Ham_Metin' sütunu çıkarıldı.")
print("📂 Dosya Adı: arcelik_master_temiz_analiz.xlsx")
print("\n--- İlk 5 Satır (Önizleme) ---")
display(output_df.head())

💾 İşlem Başarılı! 'Ham_Metin' sütunu çıkarıldı.
📂 Dosya Adı: arcelik_master_temiz_analiz.xlsx

--- İlk 5 Satır (Önizleme) ---


,Kategori,Temiz_Metin
0,Bulasik,tarihinde i̇zmir i̇stinyepark arçelik bayisinden model bulaşık makinesi satın aldıkkurulumu karabağlar servisi yaptı kurulumdan yaklaşık iki gün sonra eşim bulaşık yıkamak makinenin kapağını açtığında kapağın kısmının altında vuruk olduğunu fark etti hemen servisi aradık gelip incelediklerinde sadece değil iki adet vuruk olduğunu tespit ettiler ürünün fotoğraflarını çekip arçelik merkeze gönderdilerbiz çağrı merkezini arayıp durumu detaylı şekilde bildirerek ayıplı ürünün değiştirilmesini talep ettik yaklaşık iki gün sonra servis bizi arayıp kapağı değiştirebileceklerini garanti süresini yıldan yıla çıkarabileceklerini söylediler ben teklifi kabul etmeyince tüketici hakem heyetine başvurabileceğimizi ifade ettiler buna rağmen talebimizin ürün değişimi olduğunu açıkça belirtmemize karşın günlük beklemenin ardından ayıplı ürünü değiştirmeyi kabul etmedikleri tarafıma bildirildikapağın kısmındaki vurukların bizim kullanımımızla oluşmasının mümkün olmadığını ancak dışarıdan sert darbeyle adeta çekiçle vurulmuş hasar olduğunu düşünüyorum nedenle ürünü baştan itibaren ayıplı olarak değerlendiriyorum yeni hatasız bulaşık makinesi değişimini talep ediyorum numaralı servis kaydımın incelenerek yazılı kalıcı çözüm sunulmasını ayıplı ürünün yenisiyle değiştirilmesini istiyorum
1,Bulasik,arçelik uhı model bulaşık makinesini yaklaşık yıldır kullanıyoruz ürünümüz hâlâgarantikapsamında tarihinde makine aniden durdu çalışmıyor fişi çekilmiş şekilde bekliyor bunun üzerine arçelik servisini aradığımızda bize ancak tarihi saat randevu verilebileceği söylendii̇ki gün sonrasına randevu verilmesini arçelik firmaya yakıştıramadım bulaşık makinesinin hayatımızdaki önemini içinde bulunduğumuz mağduriyeti dikkate aldığımızda verilen hizmeti yeterli bulmuyoruz süreçte gerçekten zor durumda kaldık yaklaşım nedeniyle arçelik markasından alışveriş yapmayı düşünmüyoruzyaşadığımız mağduriyetin dikkate alınmasını servis randevu süreçlerinizin gözden geçirilmesini benzer durumlarda müşterilerin şekilde bekletilmemesini talep ediyorum
2,Bulasik,arçelik çamaşır makinemde yaklaşık aydır kapaktan kaçırma sorunu yaşıyorum makine çalışırken kapağın etrafından sızıyor etrafı ıslatıyorürünüm garanti kapsamında olmasına rağmen servis gelmeden önce benden ücret talep edildi sorunun garanti dışı olduğunu söylediler garanti devam ederken makine incelenmeden şekilde ücret istenmesini doğru bulmuyorumarızanın garanti kapsamında değerlendirilerek çamaşır makinemin yerinde kontrol edilmesini kaçağı probleminin benden hiçbir ücret talep edilmeden ücretsiz olarak onarılmasını talep ediyorum
3,Bulasik,arçelik’ten kasım ’de eskişehir tepebaşı’nda bulunan torunlar dayanıklı tüketim malları pazarlama bayisinden arçelik inox programlı bulaşık makinesi satın cihaz dolmadan ocak ’de ilk arızasını verdi paneli değişti tarihten sonra defalarca aynı arızayı verdi paneli ana kartı birçok değişti servisle adeta sürekli görüşür hale geldimyıllar içinde makine sürekli arıza verdi servis defalarca eve geldi hatta defasında makineyi alıp yaklaşık hafta serviste tuttular buna rağmen sorun tamamen çözülmediği değişim talebinde bulundum fakat talebim reddedildi yıllarına ait arıza servis kayıtlarım mevcutaralık ’te makine tekrar arızalandı panel yine değişti son servis ziyaretinde görevli cihazın temmuz ayında yan duvarlarının panelinin sacının değiştiğini makinenin önce ciddi işlem gördüğünü söyledi ben bilgiyi ürünü satın aldıktan tam yıl sonra servis elemanından öğrendim bana geçmişte büyük işlem görmüş ayıplı ürün yeni cihaz satılmışşu elimde geçmiş arıza tamirlerine dair ekran görüntüleri kayıt numaraları bulunuyor bunlardan birinin numarasıyla tekrar arıza kaydı açıldı buna rağmen arçelik arıza kayıtlarına nedense ulaşamadığını söylüyor tarafıma hâlâ net dönüş yapılmadımakinenin garantisini uzatmıştım garanti kasım ayına kadar devam ediyor yıldır sürekli arıza veren sonradan eski işlem görmüş olduğu ortaya çıkan cihazı kullanmak istemiyorum bana ayıplı mal sa